In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np
import faiss
from pathlib import Path
import os
import re
import pickle
from tqdm import tqdm

c:\Users\IshaniArya\anaconda3\envs\alo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loading in markdown and chunks for testing

#This is for sanity testing, ignore this file v
markdown_path = Path("./postgre_chunks/accumulate.md")
markdown_text = markdown_path.read_text(encoding="utf-8")

#This is what we are using
chunks_path = Path("./postgre_chunks")
chunk_texts = []

id_count = 0

for file_path in chunks_path.glob("*.txt"):
    text = file_path.read_text(encoding="utf-8")
    chunk_texts.append({
        "id": id_count,
        "type" : "code",
        "content": text.strip()
    })
    id_count += 1

print(f"Loaded {len(chunk_texts)} chunks")


Loaded 4 chunks


In [1]:
emb_models = ["nomic-ai/nomic-embed-text-v1", "jinaai/jina-embeddings-v2-small-en"]
model = SentenceTransformer(
    emb_models[0],
    trust_remote_code=True
)

def embed_texts(texts: list[str]) -> np.ndarray:
    vectors = model.encode(texts, normalize_embeddings=True, convert_to_numpy=True)
    return vectors

# Do the embedding
all_texts = [chunk["content"] for chunk in chunk_texts]
all_embeddings = embed_texts(all_texts)

NameError: name 'SentenceTransformer' is not defined

In [4]:
pip install einops

Note: you may need to restart the kernel to use updated packages.


In [ ]:
all_embeddings.shape

(4, 768)

In [ ]:
import numpy as np
import json

In [ ]:
vectors_np=np.array(all_embeddings).astype("float32")
dimension = vectors_np.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(vectors_np)

metadata_map = {i: chunk_texts[i] for i in range(len(chunk_texts))}

faiss.write_index(index, "rag_index.faiss")
with open("metadata_map.json","w") as f:
    json.dump(metadata_map, f)

print("embeddings stored in faiss, metadata saved.")    

embeddings stored in faiss, metadata saved.


In [ ]:
# Load index
index = faiss.read_index("rag_index.faiss")
 
# Load metadata
with open("metadata_map.json", "r") as f:
    metadata_map = json.load(f)
 
# Embed a query
query = "How is data cleaned?"
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")
 
# Search top-5
D, I = index.search(query_vec, k=5)
 
# Lookup original chunks
for idx in I[0]:
    chunk = metadata_map[str(idx)]
    print(f"\n[{chunk['type']}] → {chunk['content']}")

In [2]:
def build_prompt(top_chunks, user_query):
    system_prompt = "You are a helpful assistant that answers technical questions using code and documentation context.\n\n"
    context = "Context:\n"
    for i, chunk in enumerate(top_chunks):
        context += f"[{chunk['type'].upper()} CHUNK {i+1}]\n{chunk['content']}\n\n"
    return system_prompt + context + f"Question: {user_query}"


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"  # 'cuda' or 'cpu'
)

streamer = TextStreamer(tokenizer)

def get_local_llm_response(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=False,
        streamer=streamer
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
401 Client Error. (Request ID: Root=1-684b5270-6071b8433c5436140d0a6ead;7b564a4d-3600-4159-a5b8-6d58932d5f41)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted. You must have access to it and be authenticated to access it. Please log in.